<a href="https://colab.research.google.com/github/diaskrv/deepl/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random

from scipy.sparse import csr_matrix

from sklearn.neighbors import NearestNeighbors

In [2]:
movies_info = pd.read_csv('/content/movies.csv')
ratings_info = pd.read_csv('/content/ratings.csv')

In [13]:
movies_info.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [14]:
ratings_info.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [19]:
rating_by_movie_list = ratings_info.pivot(index = 'movieId', columns = 'userId', values= 'rating')
rating_by_movie_list.fillna(0, inplace = True)
rating_by_movie_list.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
number_of_movies = ratings_info.groupby(['movieId'])['rating'].count()
number_of_users = ratings_info.groupby(['userId'])['rating'].count()

user_mask = number_of_users[number_of_users > 40].index
movie_mask = number_of_movies[number_of_movies > 10].index

rating_by_movie_list = rating_by_movie_list.loc[movie_mask, user_mask]

rating_by_movie_list.shape

(2121, 422)

In [25]:
rating_by_movie_list = rating_by_movie_list.rename_axis(None, axis = 1).reset_index()
rating_by_movie_list.head()

,movieId,1,4,5,6,7,8,9,10,11,...,600,601,602,603,604,605,606,607,608,610
0,1,4.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,0.0,...,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,5.0
1,2,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,...,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0
2,3,4.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,5,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
4,6,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,5.0


In [26]:
csr_data = csr_matrix(rating_by_movie_list.reset_index().values)

Model training

In [27]:
neighbours = 20
knn = NearestNeighbors(n_neighbors = neighbours)

knn.fit(csr_data)

NearestNeighbors(n_neighbors=20)

In [28]:
rec_number = 15
movie_search = movies_info[movies_info['title'].str.contains('Ant-Man')]
movie_search

,movieId,title,genres
8689,122900,Ant-Man (2015),Action|Adventure|Sci-Fi
9713,188301,Ant-Man and the Wasp (2018),Action|Adventure|Comedy|Fantasy|Sci-Fi


In [29]:
movie_id = rating_by_movie_list[rating_by_movie_list['movieId'] == movie_search.iloc[0]['movieId']].index[0]
movie_id

2079

In [30]:
distances, indices = knn.kneighbors(csr_data[movie_id], n_neighbors = rec_number + 1)

print(indices)
print(distances)

[[2079 2081 2078 2082 2080 2077 2083 2085 2084 2076 2086 2087 2088 2075
  2074 2073]]
[[   0.           17.37095277   19.2418814    21.02379604   21.66217902
    23.68016047   23.76446928   25.22895162   25.95187854   28.03569154
    28.71410803   29.09896905   31.591138   3755.04820209 3759.05526695
  4204.04650307]]


In [31]:
indices_list = indices.squeeze().tolist()
distances_list = distances.squeeze().tolist()

indices_distances = list(zip(indices_list, distances_list))

print(indices_distances[:5])

[(2079, 0.0), (2081, 17.370952766040208), (2078, 19.241881404893856), (2082, 21.02379604162864), (2080, 21.66217902243447)]


In [32]:
indices_distances = indices_distances[1:]
indices_distances

[(2081, 17.370952766040208),
 (2078, 19.241881404893856),
 (2082, 21.02379604162864),
 (2080, 21.66217902243447),
 (2077, 23.680160472429236),
 (2083, 23.76446927663229),
 (2085, 25.228951623085727),
 (2084, 25.95187854472196),
 (2076, 28.035691537752374),
 (2086, 28.71410803072246),
 (2087, 29.098969053902923),
 (2088, 31.591137997862628),
 (2075, 3755.0482020874247),
 (2074, 3759.055266952057),
 (2073, 4204.046503072962)]

In [33]:
rec_list = []

for i in indices_distances:

    matrix_movie_id = rating_by_movie_list.iloc[i[0]]['movieId']
    id = movies_info[movies_info['movieId'] == matrix_movie_id].index
    title = movies_info.iloc[id]['title'].values[0]
    distance = i[1]

    rec_list.append({'Title' : title, 'Distance' : distance})

In [34]:
rec_table = pd.DataFrame(rec_list, index = range(1, rec_number + 1))
rec_table

,Title,Distance
1,Black Panther (2017),17.370953
2,Avengers: Age of Ultron (2015),19.241881
3,Avengers: Infinity War - Part I (2018),21.023796
4,Deadpool (2016),21.662179
5,Star Wars: Episode VII - The Force Awakens (2015),23.680160
6,Thor: Ragnarok (2017),23.764469
7,Captain America: Civil War (2016),25.228952
8,Guardians of the Galaxy 2 (2017),25.951879
9,Mad Max: Fury Road (2015),28.035692
10,Doctor Strange (2016),28.714108


In [35]:
movie_search_by_genre = movies_info[movies_info['genres'].str.contains('Western')]
movie_search_by_genre.sample(15, random_state=0)

,movieId,title,genres
8063,99114,Django Unchained (2012),Action|Drama|Western
2786,3727,Near Dark (1987),Horror|Western
5992,37211,Go West (1940),Comedy|Musical|Western
2894,3871,Shane (1953),Drama|Western
4746,7070,Red River (1948),Action|Adventure|Western
2610,3494,True Grit (1969),Adventure|Drama|Western
376,432,City Slickers II: The Legend of Curly's Gold (...,Adventure|Comedy|Western
340,383,Wyatt Earp (1994),Western
3944,5550,Love Me Tender (1956),Musical|Western
6902,63853,Australia (2008),Adventure|Drama|War|Western
